# Graph RAG for GNN Course

## 1. Установка и импорт библиотек

In [1]:
%%capture
!pip install datasets docx2txt langchain langchain_community langchain-text-splitters
!pip install torch_geometric

In [3]:
import os
import openai
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from typing import List, Dict

from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


## 2. Настройка клиента и моделей

In [4]:
client = openai.OpenAI(
    api_key="token",
    base_url="https://api.vsellm.ru/v1"
)

embed_model_name = 'openai/text-embedding-3-small'
generative_model_name = "openai/gpt-4.1-mini"

## 3. Загрузка и подготовка данных
Здесь мы загружаем документ и разбиваем его на чанки. Стратегия чанкинга немного улучшена для захвата большего контекста.

In [5]:
loader = Docx2txtLoader(
    "Курс.docx"
)
documents = loader.load()


splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=150
)

chunks = splitter.split_documents(documents)

print(f"Total chunks: {len(chunks)}")

Total chunks: 87


## 4. Генерация базовых эмбеддингов


In [6]:
def get_embeddings_batch(texts, batch_size=50):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = client.embeddings.create(
            model=embed_model_name,
            input=batch
        )
        embeddings.extend([item.embedding for item in response.data])
    return np.array(embeddings)

raw_texts = [c.page_content for c in chunks]
node_features_np = get_embeddings_batch(raw_texts)

print(f"Embeddings shape: {node_features_np.shape}")

Embeddings shape: (87, 1536)


## 5. Graph Construction & GNN Implementation

In [7]:
def build_graph_data(embeddings, k_neighbors=3, similarity_threshold=0.8):
    num_nodes = embeddings.shape[0]
    edge_index = []

    for i in range(num_nodes - 1):
        edge_index.append([i, i + 1])
        edge_index.append([i + 1, i])

    embed_tensor = torch.tensor(embeddings, dtype=torch.float)
    embed_tensor = F.normalize(embed_tensor, p=2, dim=1)

    sim_matrix = torch.mm(embed_tensor, embed_tensor.t())

    _, top_indices = torch.topk(sim_matrix, k=k_neighbors + 1, dim=1)

    top_indices = top_indices.numpy()

    for i in range(num_nodes):
        for neighbor_idx in top_indices[i]:
            if i != neighbor_idx:
                if sim_matrix[i, neighbor_idx] > similarity_threshold:
                    edge_index.append([i, neighbor_idx])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    x = torch.tensor(embeddings, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)
    return data

graph_data = build_graph_data(node_features_np, k_neighbors=4, similarity_threshold=0.75)
print(f"Graph constructed: {graph_data.num_nodes} nodes, {graph_data.num_edges} edges")

Graph constructed: 87 nodes, 266 edges


In [8]:
class GNNRefiner(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

input_dim = node_features_np.shape[1]
model = GNNRefiner(input_dim, input_dim)

with torch.no_grad():
    model.conv1.lin.weight.copy_(torch.eye(input_dim))
    model.conv2.lin.weight.copy_(torch.eye(input_dim))

model.eval()

with torch.no_grad():
    refined_embeddings = model(graph_data.x, graph_data.edge_index)
    refined_embeddings_np = refined_embeddings.numpy()

refined_embeddings_np = refined_embeddings_np / np.linalg.norm(refined_embeddings_np, axis=1, keepdims=True)

print("GNN propagation complete. Embeddings refined.")

GNN propagation complete. Embeddings refined.


## 6. Logic Retrieval (GNN-based)

In [9]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def retrieve_gnn(query, k=5):
    query_emb = client.embeddings.create(
        model=embed_model_name,
        input=[query]
    ).data[0].embedding
    query_emb = np.array(query_emb)

    scores = np.dot(refined_embeddings_np, query_emb)

    top_idx = np.argsort(scores)[-k:][::-1]

    results = []
    for i in top_idx:
        results.append({
            "text": chunks[i].page_content,
            "score": float(scores[i]),
            "id": i
        })

    best_match_idx = top_idx[0]
    edge_index = graph_data.edge_index.numpy()
    neighbors = edge_index[1][edge_index[0] == best_match_idx]

    for n_idx in neighbors:
        if n_idx not in top_idx:
            results.append({
                "text": chunks[n_idx].page_content,
                "score": scores[n_idx],
                "id": n_idx,
                "note": "neighbor_expansion"
            })
            if len(results) >= k + 2:
                break

    return results

def build_context(retrieved_chunks):
    return "\n\n".join(
        f"[Chunk ID {c['id']}, Score={c['score']:.3f}]\n{c['text']}"
        for c in retrieved_chunks
    )

## 7. Генерация ответа и финальная функция

In [10]:
def generate_answer(question, context):
    response = client.chat.completions.create(
        model=generative_model_name,
        messages=[
            {
                "role": "system",
                "content": (
                    "Ты эксперт по Graph Neural Networks (GNN). "
                    "Отвечай на основе предоставленного контекста. "
                    "Если в контексте есть информация, используй её приоритетно."
                )
            },
            {
                "role": "user",
                "content": f"""
Контекст (извлеченный с помощью GCN RAG):
{context}

Вопрос:
{question}

Ответ (кратко и по сути):
"""
            }
        ],
        temperature=0.1
    )
    return response.choices[0].message.content

def answer(question):
    retrieved = retrieve_gnn(question, k=6)
    context = build_context(retrieved)
    ans = generate_answer(question, context)

    return {
        "question": question,
        "answer": ans,
        "retrieved_chunks": retrieved
    }

In [13]:
test_q = "Какие задачи решают GNN?"
result = answer(test_q)
print(f"Q: {result['question']}")
print(f"A: {result['answer']}")
print("\nRetrieved chunks info:", [ (r['id'], f"{r['score']:.3f}") for r in result['retrieved_chunks'] ])

Q: Какие задачи решают GNN?
A: GNN решают задачи классификации узлов, детектирования аномалий (например, мошенничества), рекомендательных систем, молекулярного моделирования (прогнозирование свойств молекул), анализа социальных сетей (кластеризация, определение лидеров мнений, прогноз трендов), а также обработку динамических и гетерогенных графов.

Retrieved chunks info: [(np.int64(3), '0.466'), (np.int64(33), '0.465'), (np.int64(32), '0.460'), (np.int64(58), '0.460'), (np.int64(57), '0.460'), (np.int64(2), '0.458'), (np.int64(4), '0.456')]


In [18]:
import pandas as pd

df_qa = pd.read_excel("QA_dataset.xlsx")

In [19]:
from tqdm import tqdm

tqdm.pandas()

df_qa["Answer"] = df_qa["Question"].progress_apply(lambda x: answer(x)['answer'])

100%|██████████| 100/100 [03:35<00:00,  2.15s/it]


In [20]:
df_qa

,Question,Answer
0,Что такое граф и как он формально задаётся?,"Граф — это структура данных, состоящая из узло..."
1,Какие основные компоненты используются для опи...,Основные компоненты для описания динамического...
2,В чём отличие snapshot-based и event-based дин...,Snapshot-based динамика моделирует состояние г...
3,Какие задачи решают динамические GNN?,"Динамические GNN решают задачи анализа графов,..."
4,Почему для динамических графов требуется памят...,Для динамических графов память узлов необходим...
...,...,...
95,Какие diffusion models применяются к графам зн...,В предоставленном контексте нет информации о к...
96,Какой tokenizer лучше использовать для Graph-RAG?,"Для Graph-RAG лучше использовать токенизатор, ..."
97,Какой benchmark является стандартом для оценки...,В предоставленном контексте нет информации о с...
98,Какие методы сжатия графов лучше подходят для ...,Для Retrieval-Augmented Generation (RAG) лучше...


In [22]:
df_qa.to_excel("answers.xlsx")